In [2]:
# prompt: Importa el arhcivo DataSet que es un xlsx

import pandas as pd

# Install openpyxl if not already installed
!pip install openpyxl

# Replace 'DataSet.xlsx' with the actual filename if different
df = pd.read_excel('DataSet.xlsx', engine='openpyxl')

# Now you can work with the DataFrame 'df'
# Example: Print the first 5 rows
print(df.head())


   Numero   Dataset de Reseñas de Call Center (200 ejemplos)
0       1   Excelente atención, me resolvieron todo rápid...
1       2                El agente fue muy amable y paciente
2       3              Me encantó el servicio, muy eficiente
3       4                        Rápido, claro y profesional
4       5                     Todo perfecto, muy recomendado


In [ ]:
# Clasificador binario de reseñas de call center (versión mejorada)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

!pip install -q openpyxl

# === 1. Cargar archivo si existe ===
try:
    df = pd.read_excel('DataSet.xlsx', engine='openpyxl')
    print("✅ Archivo Excel cargado. Columnas encontradas:", df.columns.tolist())
    df = df.iloc[:, 1:]  # ignorar columna 'Número'
    df.columns = ['text']

    # Asumimos 100 positivas + 100 negativas
    n = len(df)
    half = n // 2
    df['label'] = [1]*half + [0]*(n - half)

except:
    print("⚠️ No se encontró archivo. Se generará dataset sintético enriquecido.")

    # === 2. Dataset sintético enriquecido ===

    positivas = [
        "excelente atención", "muy buen trato", "rápido y eficiente", "resolvieron todo",
        "satisfecho con el servicio", "amables y claros", "me ayudaron mucho", "felicitaciones al agente",
        "trato profesional y educado", "todo perfecto", "me sentí escuchado", "solucionaron mi problema",
        "fueron muy atentos", "estoy contento", "resolución inmediata", "sin contratiempos", "rápido y cordial",
        "muy recomendable", "claridad total", "excelente experiencia"
    ]

    negativas = [
        "esperé demasiado", "nadie contestó", "pésimo servicio", "no resolvieron nada", "mal trato",
        "fui ignorado", "me colgaron", "demasiado lento", "inútiles", "atención deficiente",
        "me dejaron en espera", "groseros y sin ayuda", "no me dieron solución", "reclamé y no respondieron",
        "una estafa", "no se hacen responsables", "problemas constantes", "mala atención", "muy decepcionado", "odio el trato"
    ]

    texts = positivas + negativas
    labels = [1]*len(positivas) + [0]*len(negativas)

    # Aumentar a 200 con repetición aleatoria
    texts_extended = list(np.random.choice(texts, 200, replace=True))
    labels_extended = [1 if t in positivas else 0 for t in texts_extended]

    df = pd.DataFrame({'text': texts_extended, 'label': labels_extended})
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# === 3. Preprocesamiento ===
df['text'] = df['text'].astype(str).str.lower()

# === 4. Dividir datos ===
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# === 5. TF-IDF con n-grams ===
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# === 6. Entrenar modelo ===
model = LogisticRegression()
model.fit(X_train_vec, y_train)



✅ Archivo Excel cargado. Columnas encontradas: ['Numero', 'Dataset de Reseñas de Call Center (200 ejemplos)']

✅ Accuracy: 1.00

📊 Reporte de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        21

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

